In [2]:
import pandas as pd
import numpy as np
import string

In [3]:
data_train = pd.read_pickle("Data/train_data.pkl")

In [4]:
data_train.head()

,comment_text,toxic,text_clean
22689,"""::No. The reason some excellent editors leav...",0,reason excellent editor leave reasoned fully e...
154371,Comment Will it merge back into Historicity of...,0,comment merge back historicity jesus tiny part...
93418,Well then I might do that if I don't get the l...,0,well might dont get list done time although st...
31049,"Several years later, I finally add something t...",0,several year later finally add something talk ...
47906,Could you please specify the licence of the ph...,0,could please specify licence photo download se...


In [5]:
def preprocess_text(text, remove_stop = True, stem_words = False, remove_mentions_hashtags = True):
    text = text.lower()
    text = text.replace("what's", "what is")
    text = text.replace("\'ve", " have")
    text = text.replace("can't", "can not")
    text = text.replace("n't", " not")
    text = text.replace("i'm", "i am")
    text = text.replace("'re", " are")
    text = text.replace("\'d", " would")
    text = text.replace("\'ll", " will")
    text = text.replace("\n", " ")
    for c in string.punctuation:
        text = text.replace(c, "")

    return text

In [6]:
data_train['comments_cleaned'] = data_train["comment_text"].apply(preprocess_text)

In [7]:
data_train

,comment_text,toxic,text_clean,comments_cleaned
22689,"""::No. The reason some excellent editors leav...",0,reason excellent editor leave reasoned fully e...,no the reason some excellent editors leave is...
154371,Comment Will it merge back into Historicity of...,0,comment merge back historicity jesus tiny part...,comment will it merge back into historicity of...
93418,Well then I might do that if I don't get the l...,0,well might dont get list done time although st...,well then i might do that if i do not get the ...
31049,"Several years later, I finally add something t...",0,several year later finally add something talk ...,several years later i finally add something to...
47906,Could you please specify the licence of the ph...,0,could please specify licence photo download se...,could you please specify the licence of the ph...
...,...,...,...,...
153719,In the meantime I can't even edit User talk:Ro...,0,meantime cant even edit user talkrompecorporat...,in the meantime i can not even edit user talkr...
148645,""" April 2011 (UTC)\nI don't dispute that the i...",0,april 2011 utc dont dispute image add article ...,april 2011 utc i do not dispute that the imag...
113159,I agree that this article is a WP:NEOLOGISM vi...,0,agree article wpneologism violation two source...,i agree that this article is a wpneologism vio...
104771,""" These comments alone constitute evidence tha...",0,comment alone constitute evidence mediation ka...,these comments alone constitute evidence that...


In [8]:
data_train[data_train.comments_cleaned == 'some jerkoff blocked me']

,comment_text,toxic,text_clean,comments_cleaned


In [10]:
print()

In [11]:
data_train[data_train.toxic == 1]

,comment_text,toxic,text_clean,comments_cleaned
125996,FUCK U ALL \n\nFUCK U ALL,1,fuck u fuck u,fuck u all fuck u all
111470,Mud \n\nWhat the fuck are you doing? I added ...,1,mud fuck added adjusted score mud dont undo ch...,mud what the fuck are you doing i added the...
16367,"""\n""""Vandalism""""? Sorry, mister, but I haven't...",1,vandalism sorry mister havent conducted vandal...,vandalism sorry mister but i have not conduct...
92963,I am coming to get you \n\ni know where you li...,1,coming get know live im going come rape,i am coming to get you i know where you live...
45928,If that is vandalism then any accusation of be...,1,vandalism accusation sockpuppet vandalism sock...,if that is vandalism then any accusation of be...
...,...,...,...,...
59646,If you have no interest in the whole story the...,1,interest whole story get f,if you have no interest in the whole story the...
59711,"""\n\n Fuck you \n\nRe: """"mark williams radio h...",1,fuck mark williams radio host ground zero mosq...,fuck you re mark williams radio host and ...
17384,ARE IGNORANT PENDEJO IS 185 IS NAME IS CONFIRM...,1,ignorant pendejo 185 name confirmed ok,are ignorant pendejo is 185 is name is confirm...
121173,You idiots always demand I explain myself on t...,1,idiot always demand explain talk page always d...,you idiots always demand i explain myself on t...


In [12]:
import csv
data_train['label'] = ['__label__'+str(s) for s in data_train['toxic']]
data = data_train[['comments_cleaned', 'label']]
data.to_csv('comments.txt', index=False, sep=' ', header=False,quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [13]:
data_train[['comments_cleaned', 'label']]
data_train = data_train.fillna('')

In [14]:
import fasttext

model = fasttext.train_supervised(input="comments2.txt")

In [15]:
def get_prediction(fast_out):
    return int(fast_out[0][0].split('__label__')[1])

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

##Training accuracy
preds = [get_prediction(model.predict(x)) for x in data_train['comments_cleaned']]
print('accuracy:', accuracy_score(data_train['toxic'], preds),'\n')

##Training F1-score
print('F1-score:', f1_score(data_train['toxic'], preds),'\n')

accuracy: 0.933347657543935 

F1-score: 0.4970614064716612 



In [17]:
from sklearn.metrics import classification_report

print(classification_report(data_train['toxic'], preds))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96    100993
           1       0.90      0.34      0.50     10706

    accuracy                           0.93    111699
   macro avg       0.92      0.67      0.73    111699
weighted avg       0.93      0.93      0.92    111699



In [19]:
data_test = pd.read_pickle("Data/test_data.pkl")

In [20]:
data_test['comments_cleaned'] = data_test["comment_text"].apply(preprocess_text)

In [ ]:
data_test = data_test.fillna('')

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

##test accuracy
preds = [get_prediction(model.predict(x)) for x in data_test['comments_cleaned']]
print('accuracy:', accuracy_score(data_test['toxic'], preds),'\n')

##test F1-score
print('F1-score:', f1_score(data_test['toxic'], preds),'\n')

accuracy: 0.9319017379679144 

F1-score: 0.4864524259609326 



In [22]:
from sklearn.metrics import classification_report

print(classification_report(data_test['toxic'], preds))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96     43284
           1       0.88      0.34      0.49      4588

    accuracy                           0.93     47872
   macro avg       0.91      0.67      0.72     47872
weighted avg       0.93      0.93      0.92     47872



In [23]:
data_valid = pd.read_pickle("Data/validation_data.pkl")

In [24]:
data_valid['comments_cleaned'] = data_valid["comment_text"].apply(preprocess_text)

In [25]:
data_valid = data_valid.fillna('')

In [27]:
##valid accuracy
preds = [get_prediction(model.predict(x)) for x in data_valid['comments_cleaned']]
print('accuracy:', accuracy_score(data_valid['toxic'], preds),'\n')

##valid F1-score
print('F1-score:', f1_score(data_valid['toxic'], preds),'\n')

accuracy: 0.9217856138047453 

F1-score: 0.4776617954070982 



In [28]:
from sklearn.metrics import classification_report

print(classification_report(data_valid['toxic'], preds))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96     57888
           1       0.66      0.38      0.48      6090

    accuracy                           0.92     63978
   macro avg       0.80      0.68      0.72     63978
weighted avg       0.91      0.92      0.91     63978



In [29]:
model.save_model("model.bin")

In [30]:
model.predict("someone blocked me")

(('__label__0',), array([0.57954049]))